In [365]:
import os
import cv2
import numpy as np
from random import choice

In [427]:
PAPER_BG_COLOR = [170, 180, 200] # old paper color in BGR format
BG_SIGMA = 1
CHANNELS = 3
MAX_CH_VAL = 255
BG_MASK_VAL = -1

In [428]:
# paper

def blank_image(height, width, bg_color=PAPER_BG_COLOR):
    img = np.zeros((height, width, CHANNELS), dtype=np.uint8)
    img[:, :] = bg_color
    return img


def add_noise(img, sigma=BG_SIGMA):
    height, width, _ = img.shape
    n = noise(height, width, sigma=sigma)
    img = img + n
    return img.clip(0, MAX_CH_VAL)


def noise(height, width, ratio=1, sigma=BG_SIGMA):
    mean = 0
    h = int(height / ratio)
    w = int(width / ratio)

    result = np.random.normal(mean, sigma, (w, h, CHANNELS))
    if ratio > 1:
        result = cv2.resize(result, dsize=(width, height), interpolation=cv2.INTER_LINEAR)
    return result.reshape((width, height, CHANNELS))


def texture(image, sigma=BG_SIGMA, turbulence=2):
    result = image.astype(float)
    height, width, _ = image.shape
    ratio = width
    while not ratio == 1:
        result += noise(height, width, ratio, sigma=sigma)
        ratio = (ratio // turbulence) or 1
    cut = np.clip(result, 0, MAX_CH_VAL)
    return cut.astype(np.uint8)

def get_paper(height, width, bg_color):
    return add_noise(texture(blank_image(height, width, bg_color)))

def get_paper_like(image, bg_color):
    return get_paper(image.shape[0], image.shape[1], bg_color)

def put_image_on_paper(img, paper):
    mask = np.zeros_like(img)
    mask[:, :, 0] = mask[:, :, 1] = mask[:, :, 2] = (img[:, :, 0] >= MAX_CH_VAL) * (img[:, :, 1] >= MAX_CH_VAL) * (img[:, :, 2] >= MAX_CH_VAL)
    mask = mask.astype(bool)
    return paper[:img.shape[0], :img.shape[1]] * mask + img * ~mask

PAPER = get_paper(4096, 4096, PAPER_BG_COLOR)

In [429]:
SHIFTED_ROWS_SHARE = 0.01
SHIFT_SCALE = 0.5

# print defect
def print_defect(img, scale=SHIFT_SCALE, shifted_share=SHIFTED_ROWS_SHARE):
    shifted_rows = np.random.binomial(1, shifted_share, img.shape[0])
    for i in range(img.shape[0]):
        max_shift = img.shape[1] * scale
        if shifted_rows[i] == 1:
            shift = np.random.randint(-max_shift, max_shift)
            new_row = np.zeros_like(img[i])
            if shift > 0:
                new_row[shift:] = img[i][:-shift]
            elif shift < 0:
                new_row[:shift] = img[i][-shift:]
            else:
                new_row = img[i]
            img[i] = new_row
    return img

In [430]:
def channel_shift(img, value=MAX_CH_VAL/5):
    img = img + value
    img = np.clip(img, 0, MAX_CH_VAL)
    img = img.astype(np.uint8)
    return img

def rotation(img, angle=45):
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((int(w/2), int(h/2)), angle, 1)
    img = cv2.warpAffine(img, M, (w, h), borderValue = (BG_MASK_VAL, BG_MASK_VAL, BG_MASK_VAL))
    return img

def translation(img, h_shift, w_shift):
    h, w = img.shape[:2]
    M = np.array([[1, 0, w_shift], [0, 1, h_shift]], dtype=np.float32)

    img = cv2.warpAffine(img, M, (w, h), borderValue = (BG_MASK_VAL, BG_MASK_VAL, BG_MASK_VAL))
    return img

def blur(img, shape = 10):
    return cv2.blur(img, (shape, shape))

def resize(img, scale_percent = 100):
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)
    
    img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    return img

In [438]:
# bg_coef gives transparency effect and an effect of paper pressed onto the background, especially for rough backgrounds
def put_image_on_background(img, bg, bg_coef=0):
    mask = np.zeros_like(img)
    mask[:, :, 0] = mask[:, :, 1] = mask[:, :, 2] = (img[:, :, 0] == BG_MASK_VAL) * (img[:, :, 1] == BG_MASK_VAL) * (img[:, :, 2] == BG_MASK_VAL)
    mask = mask.astype(bool)
    return bg * mask + img * ~mask * ((1 - bg_coef) +  bg_coef * bg / MAX_CH_VAL)

def make_augmentations(img, bg, bg_scale=200, img_scale=50, channel_shift_val=10, 
                       rot_angle=45, h_shift=500, w_shift=500, 
                       img_start_coords=[500, 500], blur_shape_val=5, bg_coef=0.05):
    bg = resize(bg, bg_scale)
    tmp_img = resize(img, img_scale)
    tmp_img = channel_shift(print_defect(put_image_on_paper(tmp_img, PAPER)), channel_shift_val)
    bg_mask_with_img = np.ones_like(bg) * BG_MASK_VAL
    bg_mask_with_img[img_start_coords[0]:img_start_coords[1] + tmp_img.shape[0], img_start_coords[1]:img_start_coords[1] + tmp_img.shape[1]] = tmp_img
    result = translation(bg_mask_with_img, h_shift, w_shift)
    result = rotation(result, rot_angle)
    result = blur(result, blur_shape_val)
    return put_image_on_background(result, bg, bg_coef)

In [439]:
# playground cell
bg = cv2.imread('Backgrounds/bg_light_table.png')
filename = "1.png"
filepath = 'Documents/' + filename
img = cv2.imread(filepath)
cv2.imwrite('test.png', make_augmentations(img, bg, bg_coef=0.0))

True

In [443]:
img_file_list = os.listdir('Documents/')
bg_file_list = os.listdir('Backgrounds/')

for file in img_file_list:
    img = cv2.imread('Documents/' + file)
    for i in range(10):
        bg = cv2.imread('Backgrounds/' + choice(bg_file_list))
        
        channel_shift_val = 15
        rot_angle = int(np.random .uniform(-45, 45))
        shift = 500
        h_shift = int(np.random.uniform(-shift, shift))
        w_shift = int(np.random.uniform(-shift, shift))
        blur_shape_val = np.random.randint(1, 10)
        img_scale= np.random.randint(30, 60)
        bg_scale= 200
        bg_coef = np.random.uniform(0, 0.2)

        aug_image = make_augmentations(img, bg, img_scale=img_scale, bg_scale=bg_scale, channel_shift_val=channel_shift_val, rot_angle=rot_angle, h_shift=h_shift, w_shift=w_shift, bg_coef=bg_coef)
        cv2.imwrite('Result/' + file[:-4]+ '_' + str(i) + '.png', aug_image)